# Table of Contents
 <p><div class="lev1"><a href="#The-Framingham-Heart-Study"><span class="toc-item-num">1&nbsp;&nbsp;</span>The Framingham Heart Study</a></div><div class="lev2"><a href="#Quick-Question"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Quick Question</a></div>

The Framingham Heart Study
==========================

The city of Framingham in the US was chosen to perform a study on its population to learn more about heart diseases.

It was chosen because of its appropriate size, for its stable population over time and the cooperation of its population and doctors that were willing to participate in this study in 1948.

We'll use logistic regression
to predict whether or not a patient experienced CHD
within 10 years of the first examination

In [1]:
framingham <- read.csv("framingham.csv")

In [2]:
str(framingham)

'data.frame':	4240 obs. of  16 variables:
 $ male           : int  1 0 1 0 0 0 0 0 1 1 ...
 $ age            : int  39 46 48 61 46 43 63 45 52 43 ...
 $ education      : int  4 2 1 3 3 2 1 2 1 1 ...
 $ currentSmoker  : int  0 0 1 1 1 0 0 1 0 1 ...
 $ cigsPerDay     : int  0 0 20 30 23 0 0 20 0 30 ...
 $ BPMeds         : int  0 0 0 0 0 0 0 0 0 0 ...
 $ prevalentStroke: int  0 0 0 0 0 0 0 0 0 0 ...
 $ prevalentHyp   : int  0 0 0 1 0 1 0 0 1 1 ...
 $ diabetes       : int  0 0 0 0 0 0 0 0 0 0 ...
 $ totChol        : int  195 250 245 225 285 228 205 313 260 225 ...
 $ sysBP          : num  106 121 128 150 130 ...
 $ diaBP          : num  70 81 80 95 84 110 71 71 89 107 ...
 $ BMI            : num  27 28.7 25.3 28.6 23.1 ...
 $ heartRate      : int  80 95 75 65 85 77 60 79 76 93 ...
 $ glucose        : int  77 76 70 103 85 99 85 78 79 88 ...
 $ TenYearCHD     : int  0 0 0 1 0 0 1 0 0 0 ...


We have 4240 patients and 16 variables.

We intend to perform a modelization, so we need to build first a training set and a testing set.

In [3]:
library(caTools)

In [4]:
set.seed(1000)
split = sample.split(framingham$TenYearCHD, SplitRatio=.65)  # We will put 65% of the data in the training set

In [5]:
train <- subset(framingham, split == TRUE)
test <- subset(framingham, split == FALSE)

We can now build a first logistic regressison model using the training set.

In [11]:
framinghamLog <- glm(TenYearCHD ~ ., data=train, family=binomial)
summary(framinghamLog)


Call:
glm(formula = TenYearCHD ~ ., family = binomial, data = train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.8487  -0.6007  -0.4257  -0.2842   2.8369  

Coefficients:
                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)     -7.886574   0.890729  -8.854  < 2e-16 ***
male             0.528457   0.135443   3.902 9.55e-05 ***
age              0.062055   0.008343   7.438 1.02e-13 ***
education       -0.058923   0.062430  -0.944  0.34525    
currentSmoker    0.093240   0.194008   0.481  0.63080    
cigsPerDay       0.015008   0.007826   1.918  0.05514 .  
BPMeds           0.311221   0.287408   1.083  0.27887    
prevalentStroke  1.165794   0.571215   2.041  0.04126 *  
prevalentHyp     0.315818   0.171765   1.839  0.06596 .  
diabetes        -0.421494   0.407990  -1.033  0.30156    
totChol          0.003835   0.001377   2.786  0.00533 ** 
sysBP            0.011344   0.004566   2.485  0.01297 *  
diaBP           -0.004740   0.008001  -0.592  0

It looks like male, age, prevalent stroke, total cholesterol, systolic blood pressure and glucose are significant in our model.

Cigarettes per day and prevalent hypertension are almost significant.

All the significant variables have positive coefficients, meaning that higher values for theses variables contribute to a higher probabilityof 10-year coronary heart disease.

Let's try to make some predictions on the test set now :

In [12]:
predictTest <- predict(framinghamLog, type="response", newdata=test)

We will also compute a confusion matrix with a threshold of $0.5$.

In [13]:
thresh <- .5
table(test$TenYearCHD, predictTest > thresh)

   
    FALSE TRUE
  0  1069    6
  1   187   11

We can immediately notice that our model rarely predicts true values.

This means that our model rarely predicts a 10-year CHD risk above 50%.

Let's check its accuracy.

In [15]:
accuracy <- (1069 + 11) / (1069 + 6 + 187 + 11)
print(accuracy)

[1] 0.8483896


This seems great. But let's compare this to a baseline model. We will build a simple one based on the most frequent outcome here, which is 0. That is a simpel baseline model that will always predict 0, that is no CHD.

This model would get this accuracy :

In [16]:
baseline_accuracy <- (1069 + 6) / (1069 + 6 + 187 + 11)
print(baseline_accuracy)

[1] 0.8444619


Well, our model barely beats the baseline model, which is okay but not that great.

Let's play with the threshold a little bit and compute the out-of-sample AUC.

In [17]:
library(ROCR)

Loading required package: gplots

Attaching package: 'gplots'

The following object is masked from 'package:stats':

    lowess



In [19]:
ROCRpred <- prediction(predictTest, test$TenYearCHD)
as.numeric(performance(ROCRpred, "auc")@y.values)

[1] 0.7421095

The AUC of our model is pretty good, which means it is able to differentiate well the low risk patients from the high risk patients.

## Quick Question

Using the previous confusion matrix for our logistic regression model on our test set with a threshold of 0.5,

What is the sensitivity of our logistic regression model on the test set, using a threshold of 0.5?

In [20]:
sensitivity <- 11 / (187+11)
print(sensitivity)

[1] 0.05555556


What is the specificity of our logistic regression model on the test set, using a threshold of 0.5?

In [21]:
specificity <- 1069 / (1069+6)
print(specificity)

[1] 0.9944186
